# RAG Application using Oracle 26ai and Langchain

This notebook demonstrates how to build a simple RAG application by using Oracle 26ai's vector storage capabilities.


## Install necessary packages

Before running the notebook, ensure you have the following packages installed:

* `langchain-oracledb`: Langchain integration for Oracle databases.
* `langchain-huggingface`: Langchain integration for Hugging Face embeddings.
* `sentence-transformers`: For generating text embeddings.
* `python-dotenv`: To manage environment variables securely.

In [ ]:
!python -m pip install -U langchain-oracledb langchain-huggingface sentence-transformers python-dotenv 

In [17]:
import os
import oracledb
from dotenv import load_dotenv
from langchain_oracledb.vectorstores import oraclevs
from langchain_oracledb.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings

## Setting up the database connection

For this notebook to work, we need to have the following environment variables set in a `.env` file or in your system environment:

* `ORACLE_USERNAME`: Your Oracle database username.
* `ORACLE_PASSWORD`: Your Oracle database password.

You also need to store the Oracle Wallet files and reference its location when making the connection. You'll download the wallet from your Oracle Cloud account.

In [18]:
load_dotenv()

WALLET_DIRECTORY = "../.wallet"
WALLET_PASSWORD = os.getenv("ORACLE_PASSWORD")

connection = oracledb.connect(
    user=os.getenv("ORACLE_USERNAME"),
    password=os.getenv("ORACLE_PASSWORD"),
    dsn="sample_low",
    config_dir=WALLET_DIRECTORY,
    wallet_location=WALLET_DIRECTORY,
    wallet_password=WALLET_PASSWORD,
)

## Preparing the knowledge base

We want to create a knowledge base that the RAG application can query. For this example, we'll use a simple list of documents.

We'll use Langchain's `Document` class to represent each document along with its metadata.

In [19]:
documents_json_list = [
    {
        "id": "creating-flows",
        "text": "Metaflow follows the dataflow paradigm which models a program as a directed graph of operations. This is a natural paradigm for expressing data processing pipelines, machine learning in particular. We call the graph of operations a flow. You define the operations, called steps, which are nodes of the graph and contain transitions to the next steps, which serve as edges. Metaflow sets some constraints on the structure of the graph. For starters, every flow needs a step called start and a step called end. An execution of the flow, which we call a run, starts at start. The run is successful if the final end step finishes successfully. What happens between start and end is up to you. You can construct the graph in between using an arbitrary combination of the following three types of transitions supported by Metaflow:",
        "link": "https://docs.metaflow.org/metaflow/basics",
    },
    {
        "id": "authoring-flows",
        "text": "With Metaflow, you might start with a simple stub, perhaps just a step to load data, and then gradually add more @steps, say, for data transformation, model training, and beyond, testing the flow at each iteration. To enable a smooth development experience, these iterations should run quickly, with minimal waiting - much like the familiar workflow in a notebook, where you build results one cell at a time.",
        "link": "https://docs.metaflow.org/metaflow/authoring-flows/introduction",
    },
    {
        "id": "running-flows",
        "text": "The Runner API allows you to start and manage Metaflow runs and other operations programmatically, for instance, to run flows in a script. The Runner class exposes a blocking API, which waits for operations to complete, as well as a non-blocking (asynchronous) APIs, prefixed with async which execute operations in the background. This document provides an overview of common patterns. For detailed API documentation, see the Runner API reference.",
        "link": "https://docs.metaflow.org/metaflow/managing-flows/runner",
    },
    {
        "id": "metaflow",
        "text": "This is another document that also covers information about the Runner API and how it exposes a blocking API.",
        "link": "https://docs.metaflow.org/",
    },
]

documents = []

for doc in documents_json_list:
    metadata = {"id": doc["id"], "link": doc["link"]}
    document = Document(page_content=doc["text"], metadata=metadata)
    documents.append(document)

## Preparing embedding model

We want to generate embeddings for our documents to store them in the Oracle vector database. For this example, we'll use a pre-trained model from Hugging Face via Langchain's `HuggingFaceEmbeddings` class.

In [20]:
model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

## Ingesting documents into Oracle

We can now ingest our documents into the Oracle vector database using a cosine similarity metric.

If this is the first time you are running this notebook, the vector store and the `vector_store` table will be created automatically.


In [21]:
vector_store = OracleVS.from_documents(
    documents,
    model,
    client=connection,
    table_name="vector_store",
    distance_strategy=DistanceStrategy.COSINE,
)

## Creating the search index

Now, we need to create the search index for the vector store.

We'll create an HNSW index with parallel 16 and Target Accuracy Specification as 97 percent.

In [23]:
oraclevs.create_index(
    connection,
    vector_store,
    params={
        "idx_name": "vector_store_hnsw_idx2",
        "idx_type": "HNSW",
        "accuracy": 97,
        "parallel": 16,
    },
)

## Querying the vector store

Finally, we can query the vector store to retrieve relevant documents based on a user's question. 

In [28]:
query = "What is exposed by the Runner API?"
result = vector_store.similarity_search(query, 2)

print("Top 2 relevant documents:\n")

for index, doc in enumerate(result):
    print(index, doc.page_content, doc.metadata)

Top 2 relevant documents:

0 This is another document that also covers information about the Runner API and how it exposes a blocking API. {'id': 'metaflow', 'link': 'https://docs.metaflow.org/'}
1 The Runner API allows you to start and manage Metaflow runs and other operations programmatically, for instance, to run flows in a script. The Runner class exposes a blocking API, which waits for operations to complete, as well as a non-blocking (asynchronous) APIs, prefixed with async which execute operations in the background. This document provides an overview of common patterns. For detailed API documentation, see the Runner API reference. {'id': 'running-flows', 'link': 'https://docs.metaflow.org/metaflow/managing-flows/runner'}
